In [1]:
import ray
from ray import tune
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

### Wine DataSet

In [2]:
X, Y = load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(178, 13) (178,)
(142, 13) (36, 13) (142,) (36,)


In [3]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    clf = MLPClassifier(solver='lbfgs', alpha=config["alpha"],
                                            hidden_layer_sizes=config["hidden_layer_sizes"],
                                            max_iter=90000,activation='relu',random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    reporter(config, mean_accuracy = accuracy_score(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))
    
tune.register_trainable("my_func", my_func)
ray.shutdown()
ray.init()

res = tune.run_experiments({
    "my_experiment": {
        "run": "my_func",
        "stop": { "mean_accuracy": 0.6 },
        "config": {
            "alpha": tune.grid_search([0.1, 0.2]),
            "hidden_layer_sizes": tune.grid_search([(5,2), (10,2)]),
        }
    }
})

c:\Users\Public\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc,alpha,hidden_layer_sizes
my_func_41690_00000,PENDING,,0.1,"(5, 2)"
my_func_41690_00001,PENDING,,0.2,"(5, 2)"
my_func_41690_00002,PENDING,,0.1,"(10, 2)"
my_func_41690_00003,PENDING,,0.2,"(10, 2)"


2022-05-10 04:31:11,549	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
2022-05-10 04:31:14,291	INFO logger.py:697 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (5, 2)}


Result for my_func_41690_00000:
  _metric:
    alpha: 0.1
    hidden_layer_sizes:
    - 5
    - 2
  date: 2022-05-10_04-31-14
  done: false
  experiment_id: 908683481dd24051bbb8433973856b5e
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.5833333333333334
  node_ip: 192.168.1.105
  pid: 14268
  time_since_restore: 0.24509358406066895
  time_this_iter_s: 0.24509358406066895
  time_total_s: 0.24509358406066895
  timestamp: 1652149874
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '41690_00000'
  
Result for my_func_41690_00000:
  _metric:
    alpha: 0.1
    hidden_layer_sizes:
    - 5
    - 2
  date: 2022-05-10_04-31-14
  done: true
  experiment_id: 908683481dd24051bbb8433973856b5e
  experiment_tag: 0_alpha=0.1,hidden_layer_sizes=(5, 2)
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.5833333333333334
  node_ip: 192.168.1.105
  pid: 14268
  time_since_restore: 0.24509358406066895
  time_this_iter_s: 0.24509358406066895
  time_tota

(pid=14268) Windows fatal exception: access violation
(pid=14268) 
2022-05-10 04:31:14,497	INFO logger.py:697 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (10, 2)}
(pid=18200) Windows fatal exception: access violation
(pid=18200) 


...

2022-05-10 04:31:15,627	INFO logger.py:697 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (5, 2)}
2022-05-10 04:31:15,656	INFO logger.py:697 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (10, 2)}


Result for my_func_41690_00001:
  _metric:
    alpha: 0.2
    hidden_layer_sizes:
    - 5
    - 2
  date: 2022-05-10_04-31-15
  done: true
  experiment_id: d91dffcf52284d798e3d25e5919d7081
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.7222222222222222
  node_ip: 192.168.1.105
  pid: 23580
  time_since_restore: 1.5964789390563965
  time_this_iter_s: 1.5964789390563965
  time_total_s: 1.5964789390563965
  timestamp: 1652149875
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '41690_00001'
  
Result for my_func_41690_00002:
  _metric:
    alpha: 0.1
    hidden_layer_sizes:
    - 10
    - 2
  date: 2022-05-10_04-31-15
  done: true
  experiment_id: efcff176a977430ba0570f82b03d823f
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.8888888888888888
  node_ip: 192.168.1.105
  pid: 23388
  time_since_restore: 1.605454444885254
  time_this_iter_s: 1.605454444885254
  time_total_s: 1.605454444885254
  timestamp: 1652149875
  timesteps_sinc

(pid=23388) Windows fatal exception: access violation
(pid=23388) 
(pid=23580) Windows fatal exception: access violation
(pid=23580) 


Trial name,status,loc,alpha,hidden_layer_sizes,acc,iter,total time (s)
my_func_41690_00000,TERMINATED,,0.1,"(5, 2)",0.583333,1,0.245094
my_func_41690_00001,TERMINATED,,0.2,"(5, 2)",0.722222,1,1.59648
my_func_41690_00002,TERMINATED,,0.1,"(10, 2)",0.888889,1,1.60545
my_func_41690_00003,TERMINATED,,0.2,"(10, 2)",0.944444,1,0.477477


2022-05-10 04:31:15,872	INFO tune.py:561 -- Total run time: 4.65 seconds (4.32 seconds for the tuning loop).
(pid=None) [2022-05-10 04:37:14,911 C 21604 4136] redis_client.cc:87:  Check failed: under_retry_limit Expected 1 Redis shard addresses, found 2
(pid=None) *** StackTrace Information ***
(pid=None) 
(pid=15548) 2022-05-10 04:37:22,665	ERROR function_runner.py:266 -- Runner Thread raised error.
(pid=15548) Traceback (most recent call last):
(pid=15548)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 260, in run
(pid=15548)     self._entrypoint()
(pid=15548)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 328, in entrypoint
(pid=15548)     return self._trainable_func(self.config, self._status_reporter,
(pid=15548)   File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\function_runner.py", line 594, in _trainable_func
(pid=15548)     output = fn()
(pid=15548)   File "<ipython-input-8-f1e274a7315c>",